# HW3 - Q3 [35 pts]

## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> add any cells to this Jupyter Notebook, because that will crash the autograder.
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> implement any additional libraries into this workbook.
</div>

All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  

<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`bucket()`

`long_trips()`

`manhattan_trips()`

`weighted_profit()`

`final_output()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from the NYC-TLC dataset during auto-grading.

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove or modify the following utility functions:
</div>

`load_data()`

`main()`

<div class="alert alert-block alert-info">
    Do <strong>not</strong> change the below cell. Run it to initialize your PySpark instance.
</div>

In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1603419756885_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-1>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remodify the below cell. It contains the function for loading data and all imports, and the function for running your code.
</div>

In [67]:
#### DO NOT CHANGE ANYTHING IN THIS CELL ####

from pyspark.sql.functions import col

def load_data(size='small'):
    # Loads the data for this question. Do not change this function.
    # This function should only be called with the parameter 'small' or 'large'
    
    if size != 'small' and size != 'large':
        print("Invalid size parameter provided. Use only 'small' or 'large'.")
        return
    
    input_bucket = "s3://fall2020-cse6242"
    
    # Load Trip Data
    trip_path = '/'+size+'/yellow_tripdata*'
    trips = spark.read.csv(input_bucket + trip_path, header=True, inferSchema=True)
    print("Trip Count: ",trips.count()) # Prints # of trips (# of records, as each record is one trip)
    
    # Load Lookup Data
    lookup_path = '/'+size+'/taxi*'
    lookup = spark.read.csv(input_bucket + lookup_path, header=True, inferSchema=True)
    
    return trips, lookup

def main(size, bucket):
    # Runs your functions implemented above.
    
    print(user())
    trips, lookup = load_data(size=size)
    trips = long_trips(trips)
    mtrips = manhattan_trips(trips, lookup)
    wp = weighted_profit(trips, mtrips)
    final = final_output(wp,lookup)
    
    # Outputs the results for you to visually see
    final.show()
    
    # Writes out as a CSV to your bucket.
    final.write.csv(bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Implement the below functions for this assignment:
<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> change any function inputs or outputs, and ensure that the dataframes your code returns align with the schema definitions commented in each function
</div>

## 3a. [1 pt] Update the `user()` function
This function should return your GT username, eg: gburdell3

In [5]:
def user():
    # Returns a string consisting of your GT username.
    return 'slad8'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3b. [2 pts] Update the `long_trips()` function
This function filters trips to keep only trips longer than 2 miles.

In [59]:
def long_trips(trips):
    # Returns a Dataframe with Schema the same as :trips:
    dftb = trips.filter(col("trip_distance") >= 2.0)
    dftb = dftb.withColumn("passenger_count", dftb["passenger_count"].cast("integer"))
    return dftb

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3c. [6 pts] Update the `manhattan_trips()` function

This function determines the top 20 locations with a `DOLocationID` in manhattan by passenger_count (pcount).

Example output formatting:

```
+--------------+--------+
| DOLocationID | pcount |
+--------------+--------+
|             5|      15|
|            16|      12| 
+--------------+--------+
```

In [46]:
def manhattan_trips(trips, lookup):
    # Returns a Dataframe with Schema: DOLocationID, pcount
    
    faun = trips.join(lookup, trips["DOLocationID"] == lookup["LocationID"], "inner").filter(col("Borough") == "Manhattan")
    faun = faun.groupBy(["DOLocationID"]).sum("passenger_count").withColumnRenamed("sum(passenger_count)", "pcount").orderBy("pcount", ascending = 0)
    
    faun = faun.limit(20)
    return faun

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3d. [6 pts] Update the `weighted_profit()` function
This function should determine the average `total_amount`, the total count of trips, and the total count of trips ending in the top 20 destinations and return the `weighted_profit` as discussed in the homework document.

Example output formatting:
```
+--------------+-------------------+
| PULocationID |  weighted_profit  |
+--------------+-------------------+
|            18| 33.784444421924436| 
|            12| 21.124577637149223| 
+--------------+-------------------+
```

In [8]:
def weighted_profit(trips, mtrips): 
    # Returns a Dataframe with Schema: PULocationID, weighted_profit
    
    # getting count of trips
    
    partAvg = trips.groupBy(["PULocationID"]).agg({"total_amount":'avg'})
    
    partDen = trips.groupBy(["PULocationID"]).count()
    
    
    partnumo = trips.groupBy(["PULocationID", "DOLocationID"]).count()
    
    balse = trips.withColumnRenamed("DOLocationID", "DOL")
    
    partncom = balse.join(mtrips, balse["DOL"] == mtrips["DOLocationID"], "right")
    
    countmse = partncom.groupBy(["PULocationID"]).count()
    
    ctmse = countmse.drop("count")
    
    
    partAvgT = partAvg.withColumnRenamed("PULocationID", "POL")
    
    partDenT = partDen.withColumnRenamed("PULocationID", "POLM")
    
    firstjoin = partAvgT.join(partDenT, partAvgT["POL"] == partDenT["POLM"], "inner").drop("POLM")
    
    ########################
    
    
    partfamm = trips.groupBy(["PULocationID", "DOLocationID"]).count()
    
    partfar = partfamm.withColumnRenamed("DOLocationID", "DOL")
    
    joined = mtrips.join(partfar, mtrips["DOLocationID"] == partfar["DOL"], "inner")
    
    
    joinCount = joined.groupBy(["PULocationID"]).sum("count")
    

    
    joiners = ctmse.join(firstjoin, ctmse["PULocationID"] == firstjoin["POL"]).drop("PULocationID")
    
    joiner = joiners.withColumnRenamed("count", "conum")
    
    joinerLast = joinCount.join(joiner, joinCount["PULocationID"] == joiner["POL"]).drop("POL")
    
    factAdd = joinerLast.withColumn("factor", joinerLast["sum(count)"]/joinerLast["conum"])
    
    mutlip = factAdd.withColumn("weighted_profit", factAdd["factor"] * factAdd["avg(total_amount)"])
    
    
    
    baller = mutlip.select(mutlip["PULocationID"], mutlip["weighted_profit"])
    
    
    
    return baller
    
 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3e. [5 pts] Update the `final_output()` function
This function will take the results of `weighted_profit`, links it to the `borough` and `zone` and returns the top 20 locations with the highest `weighted_profit`.

Example output formatting:
```
+------------+---------+-------------------+
|    Zone    | Borough |  weighted_profit  |
+----------------------+-------------------+
| JFK Airport|   Queens|  16.95897820117925|
|     Jamaica|   Queens| 14.879835188762488|
+------------+---------+-------------------+
```

In [58]:
def final_output(calc, lookup): 
    # Returns a Dataframe with Schema: Zone, Borough, weighted_profit
    
    partr = calc.join(lookup, lookup["LocationID"] == calc["PULocationID"], "inner")
    
    patty = partr.select("Zone", "Borough", "weighted_profit").orderBy("weighted_profit", ascending = 0).limit(20)
    
    return patty

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<div class="alert alert-block alert-info">
    Test your code on the small dataset first, as the large dataset will take a significantly longer time to run
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> use the same bucket url for multiple runs of the `main()` function, as this will cause errors. Make sure to change the name of your output location every time. (ie: s3://cse6242-gburdell3/output-small2)
</div>

Update the below cell with the address to your bucket, then run the below cell to run your code to store the results in S3.

When you have confirmed the results of the small dataset, run it again using the large dataset. Your output file will appear ina folder in your s3 bucket called YOUROUTPUT.csv as a csv file with a name something like part-0000-4d992f7a-0ad3-48f8-8c72-0022984e4b50-c000.csv. Download this file and rename it to q3_output.csv for submission. Do not make any other changes to the file. 

In [64]:
bucket = 's3://cse6242-slad8/output-large10'
# main('small',bucket)
main('large', bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

slad8
Trip Count:  187203269
+--------------------+-------------+------------------+
|                Zone|      Borough|   weighted_profit|
+--------------------+-------------+------------------+
|        Baisley Park|       Queens| 29.36045577913085|
|Flushing Meadows-...|       Queens|27.304845733617668|
|       South Jamaica|       Queens|26.294916239873476|
|     Randalls Island|    Manhattan|24.150989940227525|
|        Astoria Park|       Queens|21.706417112147516|
|Briarwood/Jamaica...|       Queens| 19.94506463178934|
|Springfield Garde...|       Queens|19.468309288781903|
|             Jamaica|       Queens|19.283943000137903|
|              Corona|       Queens|18.228769248155974|
|   LaGuardia Airport|       Queens|18.181338808372995|
|         Jamaica Bay|       Queens|17.100529446757896|
|             Maspeth|       Queens|17.005450640079545|
|Eltingville/Annad...|Staten Island|16.837764756944445|
|         JFK Airport|       Queens| 16.77772534824964|
|        Battery Pa

#### Testing

<div class="alert alert-block alert-info">
    You may use the below cell for any additional testing you need to do, however any code implemented below will not be run or used when grading
</div>

In [68]:
trips, lookup = load_data(size="large")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Trip Count:  187203269

In [69]:
trips = long_trips(trips)
# mtrips = manhattan_trips(trips, lookup)
# m = weighted_profit(trips, mtrips).orderBy("weighted_profit", ascending = 0)
# m.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
mtrips = manhattan_trips(trips, lookup)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
m = weighted_profit(trips, mtrips)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
w = final_output(m, lookup)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
bucket = 's3://cse6242-slad8/output-large10'
w.write.csv(bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
w.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…